In [1437]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

## Чтение данных

In [1438]:
df_amo= pd.read_csv('amocrm_sample_export.csv', encoding='utf-8')

In [1439]:
df_values= pd.read_csv('list_of_values.csv', encoding='utf-8')

In [1440]:
df_sample= pd.read_csv('certificates_list_sample.csv', encoding='utf-8')

In [1441]:
df_archive= pd.read_excel('codes_archieve1.xlsx')

C:\Users\Admin\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [1442]:
#Вывод таблицы во всю ширину
pd.set_option('display.max_columns', None)
df_amo.head(5)

,ID,Этап сделки,Бюджет,Дата создания сделки,Дата закрытия,DELIVERY_PRICE,Стоимость доставки,PRODUCTS,PRICE,SUBTOTAL,Выручка от клиента,Направление (вид сертификата),Номер сертификата,КОРНЕВАЯ СДЕЛКА,Номер 2-го сертификата,Сертификат на фотографа,Сертификат на тренера,Статус погашения,Дата оплаты подрядчику
0,26165303,Успешно реализовано,5500,30.11.2023 17:43:20,30.11.2023 18:00:00,0.0,NaN,катание на картинге (Варианты: 40 минут катани...,5500.0,5500.0,5500,Картинг,VKA4_4314611,NaN,NaN,NaN,NaN,NaN,NaN
1,26163311,Успешно реализовано,8000,30.11.2023 14:58:27,01.12.2023 15:09:00,NaN,NaN,Мотокросс и Эндуро (Варианты: Персональная тре...,8000.0,8000.0,8000,Мото Мотокросс,MT 273176,NaN,NaN,NaN,NaN,NaN,NaN
2,26162989,Успешно реализовано,5450,30.11.2023 14:33:51,01.12.2023 15:20:00,450.0,NaN,Стрельба из боевого оружия (Варианты: Курс «на...,5450.0,5000.0,5450,Тир. Стрельба из боевого оружия,GU2 21041,NaN,NaN,NaN,NaN,NaN,NaN
3,26160427,Успешно реализовано,34567,30.11.2023 11:37:23,02.12.2023 16:01:00,NaN,NaN,"Универсальный 23456 (Варианты: 34 567 руб., Бу...",34567.0,34567.0,34567,Универс. 23.456 и 34.567,WTF3 269703,NaN,NaN,NaN,NaN,NaN,NaN
4,26158865,Успешно реализовано,9000,30.11.2023 09:39:04,30.11.2023 09:47:00,NaN,NaN,Аэротруба (Варианты: 10 минут полёта (для 1-2 ...,9000.0,9000.0,9000,Аэротруба,VAT10 4353286,NaN,NaN,NaN,NaN,NaN,NaN


## Предобработка архивных префиксов

In [1443]:
df_archive.head()

,Направление,Номинал,Примечание,Префикс,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
0,Водные лыжи,водные лыжи за катером 1 час,NaN,SKI60,VSKI60,1,ЛЕТО \n05 - 10,8500,7600
1,Водные лыжи,водные лыжи за катером 40 минут,NaN,SKI40,VSKI40,2,ЛЕТО \n05 - 10,6000,5400
2,Водные лыжи,водные лыжи за катером 20 минут,NaN,SKI20,VSKI20,3,ЛЕТО \n05 - 10,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Вейксёрф и вейк за катером,вейк и сёрф за катером 1 час,NaN,WS60,VWS60,4,ЛЕТО \n05 - 10,8800,7920


In [1444]:
df_archive = df_archive.dropna(subset=['Направление','Префикс'])

In [1445]:
df_archive = df_archive.reset_index(drop=True)

In [1446]:
df_archive.rename(columns = {'Префикс':'prefix'}, inplace = True )

In [1447]:
df_archive.head()

,Направление,Номинал,Примечание,prefix,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
0,Водные лыжи,водные лыжи за катером 1 час,NaN,SKI60,VSKI60,1,ЛЕТО \n05 - 10,8500,7600
1,Водные лыжи,водные лыжи за катером 40 минут,NaN,SKI40,VSKI40,2,ЛЕТО \n05 - 10,6000,5400
2,Водные лыжи,водные лыжи за катером 20 минут,NaN,SKI20,VSKI20,3,ЛЕТО \n05 - 10,NaN,NaN
3,Вейксёрф и вейк за катером,вейк и сёрф за катером 1 час,NaN,WS60,VWS60,4,ЛЕТО \n05 - 10,8800,7920
4,Вейксёрф и вейк за катером,вейк и сёрф за катером 40 минут,NaN,WS40,VWS40,5,ЛЕТО \n05 - 10,6400,5760


In [1448]:
print(sorted(list(df_archive['prefix'].unique())))

['2WP24', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K / AT6K', 'AT6', 'AT60', 'BC1', 'BC2', 'BCR', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1+ Карта Тренер', 'CW2', 'CW2+ Карта Тренер', 'CWH', 'DC', 'DG15', 'DG20', 'DG30', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1x', 'EB2x', 'EB4x', 'ED3H', 'EDC', 'EF30', 'EF60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30 OLD', 'FM45 OLD', 'FM60 OLD', 'FMCS30 OLD', 'FMCS45 OLD', 'FMCS45R OLD', 'FMCS60 OLD', 'FMP15 OLD', 'FMP30 OLD', 'FMS30 OLD', 'FMS45 OLD', 'FMS60 OLD', 'FR', 'FR2', 'FRK', 'FROG1', 'FROG3', 'FROGM', 'FRT', 'FRTK', 'GC1', 'GC2', 'GL20', 'GL40', 'GL60', 'GR1', 'GR1 + T', 'GR2', 'GR2 + T', 'GU', 'GU2', 'HB20', 'HB30', 'HB60', 'HY30', 'HY60', 'ICE1', '

При сортировке обнаружилась кириллица в префиксах

In [1449]:
alf_ciril = 'Е Т О Р А Н К Х С В М'.split()
alf_latin = 'E T O P A H K X C B M'.split()

In [1450]:
def replace_cirillic_latin(x):
    v = x
    new_v = ''
    for i in range(len(v)):
        simv = v[i]
        if simv in alf_ciril:
            pos = alf_ciril.index(simv)
            simv = alf_latin[pos]
        new_v += simv
    return new_v

In [1451]:
df_archive['prefix'] = df_archive.apply(lambda x: replace_cirillic_latin(x['prefix']),axis=1)

Удаление OLD из префикса

In [1452]:
df1 = df_archive[df_archive['prefix'].str.contains('OLD')]

In [1453]:
df1.head()

,Направление,Номинал,Примечание,prefix,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
242,Автоспорт,OLD Контр-аварийная подготовка курс (2 дня по ...,NaN,SDC OLD,VSDC OLD,NaN,NaN,17000,13600
243,Автоспорт,OLD Контр-аварийная подготовка индивидуальное ...,NaN,SDCI OLD,VSDCI OLD,NaN,NaN,33000,???
244,САП-сёрф,OLD Сапборды на 3 часа ТОЛЬКО КАРТОЧКИ,NaN,SU3 OLD,VSU3 OLD,???,NaN,-,-
245,Самолёты,OLD STOL 30 минут для 1 человека,30 минут полёта на самолёте Внедорожник,FMS30 OLD,VFMS30 OLD,NaN,NaN,6000,4800
246,Самолёты,OLD STOL 45 минут для 1 человека,45 минут полёта на самолёте Внедорожник,FMS45 OLD,VFMS45 OLD,NaN,NaN,9000,7200


In [1454]:
df1.shape

(33, 9)

In [1455]:
df_archive.replace(r' (OLD)|(OLD) ',r'',regex=True,inplace=True)

In [1456]:
df1.head()

,Направление,Номинал,Примечание,prefix,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
242,Автоспорт,OLD Контр-аварийная подготовка курс (2 дня по ...,NaN,SDC OLD,VSDC OLD,NaN,NaN,17000,13600
243,Автоспорт,OLD Контр-аварийная подготовка индивидуальное ...,NaN,SDCI OLD,VSDCI OLD,NaN,NaN,33000,???
244,САП-сёрф,OLD Сапборды на 3 часа ТОЛЬКО КАРТОЧКИ,NaN,SU3 OLD,VSU3 OLD,???,NaN,-,-
245,Самолёты,OLD STOL 30 минут для 1 человека,30 минут полёта на самолёте Внедорожник,FMS30 OLD,VFMS30 OLD,NaN,NaN,6000,4800
246,Самолёты,OLD STOL 45 минут для 1 человека,45 минут полёта на самолёте Внедорожник,FMS45 OLD,VFMS45 OLD,NaN,NaN,9000,7200


In [1457]:
df_archive.iloc[246]['prefix']

'FMS45'

In [1458]:
df1.shape

(33, 9)

Удаление **+ Карта Тренер**, **+T**

In [1459]:
df_archive.replace(r'( *\+ *\S* *[Т|T]\S*)',r'T',regex=True,inplace=True)

In [1460]:
print(sorted(list(df_archive['prefix'].unique())))

['2SU24', '2SU60', '2WP24', 'AA', 'AA2', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K / AT6K', 'AT6', 'AT60', 'BC1', 'BC2', 'BCR', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DC', 'DG15', 'DG20', 'DG30', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1x', 'EB2x', 'EB4x', 'ED3H', 'EDC', 'EF30', 'EF60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30', 'FM45', 'FM60', 'FMCS30', 'FMCS45', 'FMCS45R', 'FMCS60', 'FMP15', 'FMP30', 'FMS30', 'FMS45', 'FMS60', 'FR', 'FR2', 'FRK', 'FROG1', 'FROG3', 'FROGM', 'FRT', 'FRTK', 'GC1', 'GC2', 'GL20', 'GL40', 'GL60', 'GR1', 'GR1T', 'GR2', 'GR2T', 'GU', 'GU2', 'HB20', 'HB30', 'HB60', 'HY30', 'HY60', 'ICE1', 'ICE1M', 'ICE2', 'ID', 'JT2', 'JT2A', 'JT3', 'JT3A

Имеются двойные префиксы MT/MT2 и MT10/MT10B

In [1461]:
archive_prefix = list(df_archive['prefix'].unique())

## Предобработка активных префиксов

In [1462]:
df_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              198 non-null    int64  
 1   Activity Name   198 non-null    object 
 2   Value           196 non-null    object 
 3   Status          198 non-null    object 
 4   Prefix          198 non-null    object 
 5   Purchase Price  167 non-null    float64
 6   Sale Price      196 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 11.0+ KB


In [1463]:
df_values.columns = df_values.columns.str.lower()

In [1464]:
df_values.columns = df_values.columns.str.strip()

In [1465]:
df_values.columns = df_values.columns.map(lambda x: re.sub( '(?<!^)(?=[ ])', '_', x ).lower())
df_values.columns = df_values.columns.map(lambda x: re.sub( ' ', '', x ))

In [1466]:
df_values.head(5)

,id,activity_name,value,status,prefix,purchase_price,sale_price
0,1,Вейксёрф и вейкборд,2 часа вейксерфинга или вейкбординга\nна волне...,on_air,WS2,NaN,22500.0
1,2,Вейксёрф и вейкборд,1 час вейксерфинга или вейкбординга\nна волне ...,on_air,WS1,NaN,11000.0
2,3,Вейксёрф и вейкборд,40 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS40,NaN,7500.0
3,4,Вейксёрф и вейкборд,30 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS30,NaN,5500.0
4,5,Вейксёрф и вейкборд,25 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS25,NaN,5000.0


In [1467]:
df_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              198 non-null    int64  
 1   activity_name   198 non-null    object 
 2   value           196 non-null    object 
 3   status          198 non-null    object 
 4   prefix          198 non-null    object 
 5   purchase_price  167 non-null    float64
 6   sale_price      196 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 11.0+ KB


In [1468]:
print(sorted(list(df_values['prefix'].unique())))

['2MWP24', '2WP24', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT6', 'AT60', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1x', 'EB2x', 'EB4x', 'ED3H', 'EDС', 'EFL30', 'EFL60', 'FR', 'FR2,', 'FRK', 'FRT', 'FRTK', 'GC1', 'GC2', 'GR1', 'GR2', 'GU', 'GU2', 'HB30', 'HB60', 'HY30', 'HY60', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KAT4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'KW', 'KW2', 'KYT', 'KYT2', 'LL1H', 'LL1HP', 'LL2H', 'LL2HP', 'LL3H', 'MB', 'MB2', 'MB2X', 'MEFL30', 'MEFL60', 'MF20', 'MF30', 'MF60', 'MHB60', 'MHY30', 'MHY60', 'MMF20', 'MMF30', 'MMF60', 'MSKI40', 'MSKI60', 'MSU24', 'MSU60', 'MT10', 'MW&F', 'MWP15', 'MWP2H', 'MWP60', 'MWS1', 'MWS2', 'MWS25', 'MWS30', 'MWS40', 'MWSN1', 'MWSN2', 'MWSN25', 'MWSN40', 'OW2', 'OWG', 'OWI', 'PL15', 'PL20', 'PL30', 'PP', '

Все prefix прописные. Имеется кириллица в префиксах

In [1469]:
df_values['prefix'] = df_values.apply(lambda x: replace_cirillic_latin(x['prefix']),axis=1)

In [1470]:
prefix_values = list(df_values['prefix'].unique())

In [1471]:
print(sorted((prefix_values)))

['2MWP24', '2WP24', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT6', 'AT60', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1x', 'EB2x', 'EB4x', 'ED3H', 'EDC', 'EFL30', 'EFL60', 'FR', 'FR2,', 'FRK', 'FRT', 'FRTK', 'GC1', 'GC2', 'GR1', 'GR2', 'GU', 'GU2', 'HB30', 'HB60', 'HY30', 'HY60', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KAT4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'KW', 'KW2', 'KYT', 'KYT2', 'LL1H', 'LL1HP', 'LL2H', 'LL2HP', 'LL3H', 'MB', 'MB2', 'MB2X', 'MEFL30', 'MEFL60', 'MF20', 'MF30', 'MF60', 'MHB60', 'MHY30', 'MHY60', 'MMF20', 'MMF30', 'MMF60', 'MSKI40', 'MSKI60', 'MSU24', 'MSU60', 'MT', 'MT10', 'MT5', 'MTT', 'MW&F', 'MWP15', 'MWP2H', 'MWP60', 'MWS1', 'MWS2', 'MWS25', 'MWS30', 'MWS40', 'MWSN1', 'MWSN2', 'MWSN25', 'MWSN40', 'OW2', 'OWG', 'OWI', 'PB', 'PB2'

In [1472]:
len(prefix_values)

197

#### Объединенный список префиксов

In [1473]:
all_prefix = list(set(prefix_values).union(set(archive_prefix)))

In [1474]:
all_prefix.sort()

In [1475]:
print(all_prefix)

['2MWP24', '2SU24', '2SU60', '2WP24', 'AA', 'AA2', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT4K / AT6K', 'AT6', 'AT60', 'AT6K', 'BC1', 'BC2', 'BCR', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DC', 'DG15', 'DG20', 'DG30', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1x', 'EB2x', 'EB4x', 'ED3H', 'EDC', 'EF30', 'EF60', 'EFL30', 'EFL60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30', 'FM45', 'FM60', 'FMCS30', 'FMCS45', 'FMCS45R', 'FMCS60', 'FMP15', 'FMP30', 'FMS30', 'FMS45', 'FMS60', 'FR', 'FR2', 'FR2,', 'FRK', 'FROG1', 'FROG3', 'FROGM', 'FRT', 'FRTK', 'GC1', 'GC2', 'GL20', 'GL40', 'GL60', 'GR1', 'GR1T', 'GR2', 'GR2T', 'GU', 'GU2', 'HB20', 'HB30', 'HB60', 'HY30', 'HY60', 'ICE1'

In [1476]:
len(all_prefix)

308

In [1477]:
archive_prefix = list(set(archive_prefix) - set(prefix_values))

In [1478]:
archive_prefix.sort()

In [1479]:
print(archive_prefix)

['2SU24', '2SU60', 'AA', 'AA2', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT4K / AT6K', 'BC1', 'BC2', 'BCR', 'DC', 'DG15', 'DG20', 'DG30', 'EF30', 'EF60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30', 'FM45', 'FM60', 'FMCS30', 'FMCS45', 'FMCS45R', 'FMCS60', 'FMP15', 'FMP30', 'FMS30', 'FMS45', 'FMS60', 'FR2', 'FROG1', 'FROG3', 'FROGM', 'GL20', 'GL40', 'GL60', 'GR1T', 'GR2T', 'HB20', 'ICE1', 'ICE1M', 'ICE2', 'ID', 'JT2', 'JT2A', 'JT3', 'JT3A', 'JT4', 'JT4A', 'JT7', 'JT7A', 'KR2', 'KR4', 'KR6', 'KVS', 'LB1X', 'LB2X', 'LB4X', 'MT/MT2', 'MT10/MT10B', 'MT10B', 'MT2B', 'MT5/MT5B', 'MT5B', 'PH', 'RS', 'SB', 'SB4', 'SBG', 'SDC', 'SDCI', 'SG7', 'SKI20', 'SN1X1', 'SN2X1', 'SU3', 'SU3H', 'TR', 'TR1', 'TR1K', 'TR2', 'TR2K', 'UC', 'UN', 'VD', 'WP20', 'WP30', 'WS20', 'WS60', 'WSN60']


In [1480]:
len(archive_prefix)

111

In [1481]:
df_amo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2622 entries, 0 to 2621
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ID                             2622 non-null   int64  
 1   Этап сделки                    2622 non-null   object 
 2   Бюджет                         2622 non-null   int64  
 3   Дата создания сделки           2622 non-null   object 
 4   Дата закрытия                  2622 non-null   object 
 5   DELIVERY_PRICE                 1647 non-null   float64
 6   Стоимость доставки             9 non-null      float64
 7   PRODUCTS                       2452 non-null   object 
 8   PRICE                          2452 non-null   float64
 9   SUBTOTAL                       2452 non-null   float64
 10  Выручка от клиента             2498 non-null   object 
 11  Направление (вид сертификата)  2622 non-null   object 
 12  Номер сертификата              2622 non-null   o

In [1482]:
df_values.head(10)

,id,activity_name,value,status,prefix,purchase_price,sale_price
0,1,Вейксёрф и вейкборд,2 часа вейксерфинга или вейкбординга\nна волне...,on_air,WS2,NaN,22500.0
1,2,Вейксёрф и вейкборд,1 час вейксерфинга или вейкбординга\nна волне ...,on_air,WS1,NaN,11000.0
2,3,Вейксёрф и вейкборд,40 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS40,NaN,7500.0
3,4,Вейксёрф и вейкборд,30 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS30,NaN,5500.0
4,5,Вейксёрф и вейкборд,25 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS25,NaN,5000.0
5,6,Вейксёрф и вейкборд,2 часа вейксерфинга или вейкбординга\nна любой...,on_air,WSN2,NaN,26500.0
6,7,Вейксёрф и вейкборд,1 час вейксерфинга или вейкбординга на любой в...,on_air,WSN1,NaN,13500.0
7,8,Вейксёрф и вейкборд,40 минут вейксерфинга или вейкбординга на любо...,on_air,WSN40,NaN,8500.0
8,9,Вейксёрф и вейкборд,25 минут вейксерфинга или вейкбординга на любо...,on_air,WSN25,NaN,6000.0
9,10,Флайборд,1 час полёта на флайборде,on_air,MF60,NaN,12500.0


## Предобработка базы AMO

### Имена столбцов

Столбцы в нижний регистр и удаление пробелов

In [1483]:
df_amo.columns = df_amo.columns.str.lower()

In [1484]:
df_amo.columns = df_amo.columns.str.strip()

In [1485]:
df_amo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2622 entries, 0 to 2621
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             2622 non-null   int64  
 1   этап сделки                    2622 non-null   object 
 2   бюджет                         2622 non-null   int64  
 3   дата создания сделки           2622 non-null   object 
 4   дата закрытия                  2622 non-null   object 
 5   delivery_price                 1647 non-null   float64
 6   стоимость доставки             9 non-null      float64
 7   products                       2452 non-null   object 
 8   price                          2452 non-null   float64
 9   subtotal                       2452 non-null   float64
 10  выручка от клиента             2498 non-null   object 
 11  направление (вид сертификата)  2622 non-null   object 
 12  номер сертификата              2622 non-null   o

Удаление пробелов и вставка символа "_"

In [1486]:
df_amo.columns = df_amo.columns.map(lambda x: re.sub( '(?<!^)(?=[ ])', '_', x ).lower())
df_amo.columns = df_amo.columns.map(lambda x: re.sub( ' ', '', x ))

Копирование исходной базы

In [1487]:
df_amo_temp = df_amo.copy()

Переименование столбцов

In [1488]:
df_amo_temp.rename(columns = {'номер_сертификата':'code', 'id':'amo_id','дата_закрытия':'sale_date','дата_оплаты_подрядчику':'card_used_date'}, inplace = True )

In [1489]:
df_values.rename(columns = {'id':'activity_value_id' }, inplace = True )

In [1490]:
df_amo_temp['code'] = df_amo_temp['code'].str.upper()

In [1491]:
df_amo_temp['номер_2-го_сертификата'] = df_amo_temp['номер_2-го_сертификата'].str.upper()

In [1492]:
df_amo_temp['сертификат_на_фотографа'] = df_amo_temp['сертификат_на_фотографа'].str.upper()

In [1493]:
df_amo_temp['сертификат_на_тренера'] = df_amo_temp['сертификат_на_тренера'].str.upper()

In [1494]:
df_amo_temp['дата_создания_сделки'] = pd.to_datetime(df_amo_temp['дата_создания_сделки'] )

In [1495]:
df_amo_temp['sale_date'] = pd.to_datetime(df_amo_temp['sale_date'])

In [1496]:
df_amo_temp['card_used_date'] = pd.to_datetime(df_amo_temp['card_used_date'],errors='coerce')

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '25.11.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18.11.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '26.11.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '27.10.2023' in DD/MM/YYYY format. Provide format or specif

In [1497]:
df_amo_temp = df_amo_temp.sort_values(by=["дата_создания_сделки"], ascending=True)

In [1498]:
df_amo_temp = df_amo_temp.reset_index(drop=True)

In [1499]:
df_amo_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2622 entries, 0 to 2621
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   amo_id                         2622 non-null   int64         
 1   этап_сделки                    2622 non-null   object        
 2   бюджет                         2622 non-null   int64         
 3   дата_создания_сделки           2622 non-null   datetime64[ns]
 4   sale_date                      2622 non-null   datetime64[ns]
 5   delivery_price                 1647 non-null   float64       
 6   стоимость_доставки             9 non-null      float64       
 7   products                       2452 non-null   object        
 8   price                          2452 non-null   float64       
 9   subtotal                       2452 non-null   float64       
 10  выручка_от_клиента             2498 non-null   object        
 11  направление_(вид_

In [1500]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
0,23580385,Успешно реализовано,5200,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание на картинге (Варианты: 40 минут катани...,5200.0,5200.0,5200,Картинг,KA4 213421,NaN,NaN,NaN,NaN,NaN,NaT
1,23594083,Успешно реализовано,4850,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы (Варианты: 1 квадрик 1 час без пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1 125315,NaN,NaN,NaN,NaN,NaN,NaT
2,23596587,Успешно реализовано,4850,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет на параплане (Варианты: 1 полет с инстру...,4850.0,4500.0,4850,Параплан,P1G 259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09
3,23596239,Успешно реализовано,6000,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк за катером (Варианты: 30 минут Mastercraf...,6000.0,6000.0,6000,Вейксёрф и вейк за катером,WS30 239328,NaN,NaN,NaN,TR1 210177,NaN,NaT
4,23598009,Успешно реализовано,5200,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание на картинге (Варианты: 40 минут катани...,5200.0,5200.0,5200,Картинг,КА4213697,NaN,NaN,NaN,NaN,NaN,NaT


Определение разделителей в поле **code**

In [1501]:
cc = df_amo_temp['code'].unique()

In [1502]:
set_cc = set()

In [1503]:
for el in cc:
    set_cc.update(el)

In [1504]:
print(set_cc)

{'W', 'R', ' ', '3', 'H', 'S', 'N', 'С', '8', 'B', '4', 'C', 'Y', 'И', 'U', 'T', '(', '2', 'M', 'K', 'L', '&', '\xa0', 'J', '9', 'Р', '0', 'I', '1', 'P', 'К', 'G', 'М', 'Т', 'F', '6', 'E', 'X', 'V', 'А', 'D', 'A', '7', '_', '5', 'Н', 'В', ')'}


В качестве разделителей используются символы ' ', '_', '\xa0' и их комбинации

В значениях поля code имеются символы кириллицы

Замена кириллицы на латинские символы

In [1505]:
#df_amo_temp['code'] = df_amo_temp.apply(replace_cirillic_latin,axis=1)
df_amo_temp['code'] = df_amo_temp.apply(lambda x: replace_cirillic_latin(x['code']),axis=1)

Замена всех возможных разделителей на '_'

In [1506]:
#df_amo_temp['code'] = df_amo_temp['code'].str.replace('[^_ \xa0]+([ _\xa0]+)\d+','_',regex=True)
df_amo_temp.replace(r'([ _\xa0]+)',r'_',regex=True,inplace=True)

In [1507]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
0,23580385,Успешно_реализовано,5200,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT
1,23594083,Успешно_реализовано,4850,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT
2,23596587,Успешно_реализовано,4850,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09
3,23596239,Успешно_реализовано,6000,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT
4,23598009,Успешно_реализовано,5200,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4213697,NaN,NaN,NaN,NaN,NaN,NaT


## Создание отдельных строк для 2-го сертификата

In [1508]:
df2sert = df_amo_temp.loc[(df_amo_temp['номер_2-го_сертификата'].notna())] 

In [1509]:
df2sert.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
23,24441731,Успешно_реализовано,8000,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,"Самолёт_Cessna_182_(Варианты:_20_минут_полёта,...",8000.0,8000.0,8000,Авиа_Самолёты,CS20_1905810537,NaN,VKA6_362697,NaN,NaN,Погашен,2023-07-21
63,23460055,Успешно_реализовано,10000,2023-01-13 11:50:38,2023-01-13 11:59:00,NaN,NaN,Заезд_на_багги_(Варианты:_Занятие_для_одного_1...,10000.0,10000.0,10000,Мото_Дрифт-трайк,VDTM_3651601,NaN,VBGH_2415955DIS,NaN,NaN,NaN,NaT
75,23476291,Успешно_реализовано,10000,2023-01-16 14:34:03,2023-01-16 16:17:00,NaN,NaN,"Снегоходы_(Варианты:_1_снежик_на_2_часа,_Бумаж...",9000.0,9000.0,9000,Снегоходы,VKVP2X1_2696914,NaN,VSN2X_2416042DIS,NaN,NaN,NaN,NaT
80,23499379,Успешно_реализовано,13750,2023-01-18 15:21:40,2023-01-18 18:43:00,0.0,NaN,Заезд_на_багги_(Варианты:_Занятие_для_детей_1_...,13750.0,13750.0,13750,Багги,VBGH_2430880,NaN,VBGKH_2416119,NaN,NaN,Погашен,2023-05-26
148,23990017,Успешно_реализовано,6000,2023-02-04 10:59:02,2023-04-04 16:25:00,0.0,NaN,"Катер_без_капитана_(Варианты:_1_час_проката,_Б...",5500.0,5500.0,6000,Катер_без_капитана_(Летолодка),VLL1H_173317,NaN,VLL1H_2818826_DIS,NaN,NaN,Погашен,2023-06-18


In [1510]:
df2sert.shape

(82, 19)

Имеются 2-е сертификаты, имеющие пометку DIS

Исключение подобных 2-х сертификатов

In [1511]:
df2sert = df2sert.loc[~df2sert['номер_2-го_сертификата'].str.contains('DI')]

In [1512]:
df2sert.shape

(49, 19)

In [1513]:
df2sert.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
23,24441731,Успешно_реализовано,8000,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,"Самолёт_Cessna_182_(Варианты:_20_минут_полёта,...",8000.0,8000.0,8000,Авиа_Самолёты,CS20_1905810537,NaN,VKA6_362697,NaN,NaN,Погашен,2023-07-21
80,23499379,Успешно_реализовано,13750,2023-01-18 15:21:40,2023-01-18 18:43:00,0.0,NaN,Заезд_на_багги_(Варианты:_Занятие_для_детей_1_...,13750.0,13750.0,13750,Багги,VBGH_2430880,NaN,VBGKH_2416119,NaN,NaN,Погашен,2023-05-26
233,23680799,Успешно_реализовано,12500,2023-02-15 11:03:35,2023-02-15 10:37:00,NaN,NaN,Искусственная_волна_Flow_Moscow_(Варианты:_1_ч...,12500.0,12500.0,12500,Искусственная_волна,VCW1T_2557338,NaN,VCW1T_2516846,NaN,NaN,Погашен,2023-03-17
294,23719721,Успешно_реализовано,6000,2023-02-21 16:09:36,2023-02-22 12:03:00,NaN,NaN,Подарочный_сертификатна_вейксёрфинг_(Выберите_...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS40_210129289,NaN,VWS40_2517352,NaN,NaN,Погашен,2023-07-22
409,23778849,Успешно_реализовано,6500,2023-03-03 18:55:38,2023-03-03 19:04:00,NaN,NaN,Искусственная_волна_Flow_Moscow_(Варианты:_1_ч...,6500.0,6500.0,6500,Искусственная_волна,CW1_269598,NaN,TR1_212055,NaN,NaN,Погашен,2023-04-17


In [1514]:
def insert_row_sert2(row):
    df_amo_temp.loc[len(df_amo_temp.index)] = [row['amo_id'],
                                           np.NaN,
                                          np.NaN,
                                           row['дата_создания_сделки'],
                                           row['sale_date'],
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          '',
                                          row['номер_2-го_сертификата'],
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN]
    return 1
    

In [1515]:
df_amo_temp.shape

(2622, 19)

In [1516]:
df2sert['qq']= df2sert.apply(insert_row_sert2,axis=1)

In [1517]:
df_amo_temp.shape

(2671, 19)

In [1518]:
df_amo_temp.query('amo_id == 24441731')

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
23,24441731,Успешно_реализовано,8000.0,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,"Самолёт_Cessna_182_(Варианты:_20_минут_полёта,...",8000.0,8000.0,8000,Авиа_Самолёты,CS20_1905810537,NaN,VKA6_362697,NaN,NaN,Погашен,2023-07-21
2622,24441731,NaN,NaN,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,,VKA6_362697,NaN,NaN,NaN,NaN,NaN,NaT


В базу AMO добавлены строки для вторых сертификатов

### Формирование новых столбцов в базе АМО

In [1519]:
df_amo_temp['id'] = ''

#### prefix and number

In [1520]:
df_amo_temp['prefix'] = df_amo_temp['code'].apply(lambda x: re.split('[ _]',x)[0] if ' ' in x or '_' in x else '')

In [1521]:
df_amo_temp['number'] = df_amo_temp['code'].apply(lambda x: re.split('[ _]',x)[1] if ' ' in x or '_' in x else '')

In [1522]:
df_amo_temp.head(10)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4213697,NaN,NaN,NaN,NaN,NaN,NaT,,,
5,23766769,Успешно_реализовано,6350.0,2023-01-03 12:02:45,2023-02-03 20:30:00,350.0,NaN,Заезд_на_Дрифт_Трайке_(Варианты:_Заезд_40_мину...,6350.0,6000.0,6350,Мото_Дрифт-трайк,DT40_285464,NaN,NaN,NaN,NaN,NaN,NaT,,DT40,285464
6,23695437,Успешно_реализовано,10000.0,2023-01-03 14:55:47,2023-02-17 15:03:00,0.0,NaN,Фридайвинг_(Варианты:_Индивидуальная_тренировк...,10000.0,10000.0,10000,Фридайвинг,VFR2655363,NaN,NaN,NaN,NaN,NaN,NaT,,,
7,23770797,Успешно_реализовано,0.0,2023-01-03 16:05:22,2023-01-03 16:06:00,NaN,NaN,NaN,NaN,NaN,NaN,Фридайвинг,VFR2666164,#23769895,NaN,NaN,NaN,Погашен,2023-10-07,,,
8,23772175,Успешно_реализовано,9350.0,2023-01-03 17:59:38,2023-04-03 17:35:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,9350.0,9000.0,9350,Квадроциклы,KV1X1_162569,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,162569
9,23774335,Успешно_реализовано,6000.0,2023-01-03 22:16:49,2023-02-03 10:20:00,0.0,NaN,"Бигфуты_(Варианты:_40_мин._для_1_с_пассажиром,...",6000.0,6000.0,6000,Квадроциклы_Бигфуты,VBF40_2683331,NaN,NaN,NaN,NaN,NaN,NaT,,VBF40,2683331


In [1523]:
prefix_amo = list(df_amo_temp['prefix'].unique())
prefix_amo.sort()
prefix_amo = prefix_amo[1:]

In [1524]:
print(prefix_amo)

["'+", 'AC60', 'AT10', 'AT15', 'AT4', 'AT4K', 'AT6', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'CS20', 'CS30', 'CS40', 'CS60', 'CW1', 'CW2', 'DRC2H', 'DT230', 'DT240', 'DT30', 'DT40', 'DT60', 'EB1X', 'EB2X', 'ED3H', 'EDC', 'EF30', 'EFL30', 'EFL60', 'ES130', 'F1', 'GC1', 'GC2', 'GR1', 'GR1X1', 'GR2', 'GU', 'GU2', 'HB20', 'HB30', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KR2', 'KR4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'LL1H', 'LL2H', 'MF', 'MF20', 'MF30', 'MF60', 'MT', 'MT2', 'MT5', 'MTT', 'P1', 'P1G', 'P1P', 'P2', 'P2G', 'PB', 'PB2', 'PB2X', 'PPP', 'PR20', 'PR30', 'PR40', 'PR60', 'PS', 'RB15', 'RB20', 'RB30', 'RC', 'RCV', 'RJ', 'RJ2', 'RTV', 'RXV', 'S', 'S2', 'SC', 'SC7', 'SG7', 'SH', 'SH2', 'SHH', 'SKI40', 'SKIN60', 'SKT1', 'SKT2', 'SKV', 'SN1X', 'SN1X1', 'SN2X', 'SN2X1', 'SP', 'SP2', 'ST', 'SU24', 'SU60', 'TR', 'TR1', 'TR2', 'TR2K', 'UN', 'VAC30', 'VAC60', 'VAT10', 'VAT15', 'VAT30', 'VAT4', 'VAT4K', 'VAT6', 'VAT6K', 'VBF180', 'VBF40', 'VBF90', 'VBG', 'VBGH', 'VBGK', 'VB

Некоторые значения в code без разделителей, потому не выделились prefix и number

Отделение prefix от number

In [1525]:
def separate_prefix_number(row):
    pl = prefix_amo
    v = row['code']
    if row['prefix'] == '':
        amo_id = row['amo_id']
        
        for el in pl:
            if el in row['code']:
                try:
                    v = el + '_' + row['code'][len(el):] 
                except:
                    pass
                finally:
                    return v
                
        for el in all_prefix:
            if el in row['code']:
                try:
                    v = el + '_' + row['code'][len(el):] 
                except:
                    pass
                finally:
                    return v
    return v

In [1526]:
df_amo_temp['code'] = df_amo_temp.apply(separate_prefix_number,axis=1)

In [1527]:
df_amo_temp.head(5)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213697,NaN,NaN,NaN,NaN,NaN,NaT,,,


In [1528]:
df_amo_temp.loc[df_amo_temp['prefix'] == '','prefix'] = \
df_amo_temp['code'].apply(lambda x: re.split('[_]',x)[0] if ' ' in x or '_' in x else '')

In [1529]:
df_amo_temp.loc[df_amo_temp['number'] == '','number'] = \
df_amo_temp['code'].apply(lambda x: re.split('[_]',x)[1] if ' ' in x or '_' in x else '')

In [1530]:
df_amo_temp.head(5)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213697,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213697


База АМО без prefix

In [1531]:
df_amo_without_prefix = df_amo_temp.loc[df_amo_temp['prefix'] == '']

In [1532]:
df_amo_without_prefix.shape

(7, 22)

In [1533]:
df_amo_without_prefix

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
239,23691835,Успешно_реализовано,5000.0,2023-02-16 20:33:34,2023-02-17 23:16:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,5000,Параплан,80688,NaN,NaN,NaN,NaN,NaN,NaT,,,
554,23959571,Успешно_реализовано,5600.0,2023-03-28 19:29:43,2023-03-28 22:01:00,NaN,NaN,Аэротруба_(Варианты:_6_минут_полёта_(для_1-2_ч...,5600.0,5600.0,5600,Аэротруба,VAT37112,NaN,NaN,NaN,NaN,NaN,NaT,,,
593,24217649,Успешно_реализовано,2000.0,2023-04-05 13:54:40,2023-04-05 14:24:00,0.0,NaN,Подарочный_сертификатна_сап-сёрфинг_(Выберите_...,2000.0,2000.0,2000,САП-сёрф,29510,NaN,NaN,NaN,NaN,NaN,NaT,,,
679,24084549,Успешно_реализовано,5700.0,2023-04-15 21:38:45,2023-01-05 16:47:00,500.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5700.0,5200.0,NaN,Картинг,75453,NaN,NaN,NaN,NaN,NaN,NaT,,,
1591,24816735,Успешно_реализовано,0.0,2023-07-15 09:04:30,2023-07-17 12:46:00,NaN,NaN,NaN,NaN,NaN,0,Багги,19529,#24815607,BG_220721,NaN,NaN,NaN,NaT,,,
2258,23443225,Успешно_реализовано,0.0,2023-10-01 18:29:34,2023-10-01 18:29:00,NaN,NaN,NaN,NaN,NaN,NaN,Вейк-парк,VWP62133,NaN,NaN,NaN,NaN,NaN,NaT,,,
2558,23456591,Успешно_реализовано,10500.0,2023-12-01 17:38:01,2023-01-14 05:22:00,500.0,NaN,Подарочный_сертификатв_вейк-парк_(Выберите_тип...,10500.0,10000.0,10500,Вейк-парк,2WP89087,NaN,NaN,NaN,NaN,NaN,NaT,,,


In [1534]:
#df_amo_temp = df_amo_temp.loc[df_amo_temp['prefix'] != '']

In [1535]:
df_amo_temp['card_type'] ='paper'

In [1536]:
#df_amo_temp['series'] =''

In [1537]:
df_amo_temp['design_type'] = 'design1'

In [1538]:
df_amo_temp['status'] = ''

In [1539]:
df_amo_temp['stock_status'] = ''

In [1540]:
df_amo_temp['sale_price'] = 0

In [1541]:
df_amo_temp['expiry_date'] = ''

In [1542]:
df_amo_temp['created_at'] = ''

In [1543]:
df_amo_temp['updated_at'] = ''

In [1544]:
df_amo_temp['deleted_at'] = ''

In [1545]:
df_amo_temp.loc[(df_amo_temp['card_used_date'].notna()) |  (df_amo_temp['статус_погашения'] == 'Погашен'),'status'] = 'sold_used'

In [1546]:
df_amo_temp.query('status=="sold_used"').head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395,paper,design1,sold_used,,0,,,,
7,23770797,Успешно_реализовано,0.0,2023-01-03 16:05:22,2023-01-03 16:06:00,NaN,NaN,NaN,NaN,NaN,NaN,Фридайвинг,VFR_2666164,#23769895,NaN,NaN,NaN,Погашен,2023-10-07,,VFR,2666164,paper,design1,sold_used,,0,,,,
11,23985509,Успешно_реализовано,14000.0,2023-01-04 12:26:13,2023-03-04 11:50:00,700.0,NaN,Вертолет_Robinson_R44_(Варианты:_15_минут_полё...,13200.0,12500.0,13200,Авиа_Вертолет,RB15_166049,NaN,NaN,NaN,NaN,Погашен,2023-08-25,,RB15,166049,paper,design1,sold_used,,0,,,,
14,23988163,Успешно_реализовано,2600.0,2023-01-04 20:35:41,2023-01-04 20:43:00,0.0,NaN,Катание_на_картинге_(Варианты:_20_минут_катани...,2600.0,2600.0,2600,Картинг,VKA2_2890897,NaN,NaN,NaN,NaN,Погашен,2023-02-04,,VKA2,2890897,paper,design1,sold_used,,0,,,,
15,24191536,Успешно_реализовано,6000.0,2023-01-05 01:03:54,2023-01-05 15:49:00,0.0,NaN,Вейк-сёрфинг_за_катером_(Варианты:_30_минут_Ma...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,VWS30T_2987448,NaN,NaN,NaN,NaN,Погашен,2023-07-22,,VWS30T,2987448,paper,design1,sold_used,,0,,,,


### Обработка поля **status**

In [1547]:
df_amo_temp.loc[(df_amo_temp['card_used_date'].notna()) |  \
                (df_amo_temp['статус_погашения'] == 'Погашен'),'status'] = 'sold_used'

### Обработка **sale_price**

#### Строки, являющиеся корневыми сделками

In [1548]:
lst_root_deal = list(df_amo_temp.query('корневая_сделка.notna()')['корневая_сделка'])

In [1549]:
print(lst_root_deal)

['#23769895', '#24693067', '#24964891', '#23459729', '1715029001', '#23586703', '#23514709', '#24696037', '#24696037', '#24975785', '#23681147', '#23681147', '#23705733', '#23703937', '#23710155', '#23716727', '#23716727', '#23716727', '23717485', '23718601', '23718601', '23718601', '#23724197', '#23723181', '23727459', '#23728823', '#23729565', '#23729565', '#23729565', '23732757', '#23720975', '#23748665', '#23756947', '#24204699', '#24715547', '#25885733', '#23906061', '#23918309', '#23918309', '23923847', '23923847', '#23931737', '23963847', '#23968933', '#23965909', '#23772175', '24214499', '24723003', '24992009', '#25228541', '#25538207', '#25538207', '#24072407', '#24083997', '#24098553', '#24103527', '24108151', '#24122097', 'КR2_230212', '#24177302', '#23787591', '#24996157', '24292405', '#24294657', '#24308743', '24349847', '24378991', '#24386001', '24402725', '24403171', '24403285', '24432999', '24434571', '24436793', '24437073', '#23624295', '#23624295', '#23624295', '24740

In [1550]:
len(lst_root_deal)

135

In [1551]:
def clear_num(num):
    """
    Очистка id корневой сделки и перевод ео в int
    """
    if num.isdigit():
        return int(num)
    else:
        l = len(num)-1
        while l>0:
            if num[l].isdigit():
                l -= 1
            else:
                break
        return int(num[l+1:])

Проверка:

In [1552]:
print(clear_num('КR2_230212'))

230212


In [1553]:
lst_root_deal = list(map(clear_num,lst_root_deal))

Список id строк, являющимися корневыми сделками

In [1554]:
print(lst_root_deal)

[23769895, 24693067, 24964891, 23459729, 1715029001, 23586703, 23514709, 24696037, 24696037, 24975785, 23681147, 23681147, 23705733, 23703937, 23710155, 23716727, 23716727, 23716727, 23717485, 23718601, 23718601, 23718601, 23724197, 23723181, 23727459, 23728823, 23729565, 23729565, 23729565, 23732757, 23720975, 23748665, 23756947, 24204699, 24715547, 25885733, 23906061, 23918309, 23918309, 23923847, 23923847, 23931737, 23963847, 23968933, 23965909, 23772175, 24214499, 24723003, 24992009, 25228541, 25538207, 25538207, 24072407, 24083997, 24098553, 24103527, 24108151, 24122097, 230212, 24177302, 23787591, 24996157, 24292405, 24294657, 24308743, 24349847, 24378991, 24386001, 24402725, 24403171, 24403285, 24432999, 24434571, 24436793, 24437073, 23624295, 23624295, 23624295, 24740295, 24548805, 24579543, 24579543, 24579543, 24579543, 24586559, 24606011, 24621605, 24646105, 24651727, 24669423, 24676465, 24678867, 24029505, 24030407, 25010665, 25010665, 25010665, 25011851, 25022279, 24815607,

In [1555]:
df_amo_temp.query('amo_id in @lst_root_deal').head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
8,23772175,Успешно_реализовано,9350.0,2023-01-03 17:59:38,2023-04-03 17:35:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,9350.0,9000.0,9350,Квадроциклы,KV1X1_162569,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,162569,paper,design1,,,0,,,,
40,24693067,Успешно_реализовано,8000.0,2023-01-07 18:27:21,2023-01-07 18:44:00,0.0,NaN,"Роупджампинг_(Варианты:_Одиночный_прыжок,_Бума...",8000.0,8000.0,8000,Роупджампинг,VRJ_3466463,NaN,NaN,NaN,NaN,NaN,NaT,,VRJ,3466463,paper,design1,,,0,,,,
61,23459729,Успешно_реализовано,11000.0,2023-01-13 11:16:07,2023-01-13 11:44:00,NaN,NaN,Искусственная_волна_Flow_Moscow_(Варианты:_1_ч...,11000.0,11000.0,11000,Искусственная_волна,VCW1_2484684,NaN,NaN,NaN,NaN,Погашен,2023-06-29,,VCW1,2484684,paper,design1,sold_used,,0,,,,
131,23514709,Успешно_реализовано,19900.0,2023-01-29 22:13:18,2023-01-31 01:27:00,NaN,NaN,Школа_дрифта_(Бумажный_или_электронный:_Электр...,19900.0,19900.0,19900,Авто_Дрифт,VDRC2H_2534706,NaN,NaN,NaN,NaN,Погашен,2023-09-02,,VDRC2H,2534706,paper,design1,sold_used,,0,,,,
133,23586703,Успешно_реализовано,16450.0,2023-01-31 11:09:39,2023-01-31 17:52:00,450.0,NaN,Мотокросс_и_Эндуро_(Варианты:_Персональная_тре...,16450.0,16000.0,16450,Мото_Мотокросс,MT2_184809,NaN,NaN,NaN,NaN,Погашен,2023-03-05,,MT2,184809,paper,design1,sold_used,,0,,,,


Если в сделке 1 товар и сделка не имеет производных сделок, то использовать значение поля SUBTOTAL

#### Расчет sale_price для обычных товаров

In [1556]:
df_amo_temp.loc[(df_amo_temp['номер_2-го_сертификата'].isna()) & \
                (df_amo_temp['сертификат_на_тренера'].isna()) &\
                (df_amo_temp['сертификат_на_фотографа'].isna()) &\
                (~df_amo_temp['amo_id'].isin(lst_root_deal)) &\
                (df_amo_temp['корневая_сделка'].isna()),\
                'sale_price'] = df_amo_temp['subtotal']

Проверка

Сумма поля sale_price для сделок, являющихся корневыми

In [1557]:
df_amo_temp.query('amo_id in @lst_root_deal')['sale_price'].sum()

0.0

Сумма поля sale_price для сделок, не являющихся корневыми, не имеющих 2 серт, не имеющих серт на тренера и фотографа

In [1558]:
df_amo_temp.query('amo_id in @lst_root_deal')['subtotal'].sum()

1360439.0

In [1559]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421,paper,design1,,,5200.0,,,,
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315,paper,design1,,,4500.0,,,,
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395,paper,design1,sold_used,,4500.0,,,,
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328,paper,design1,,,0.0,,,,
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213697,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213697,paper,design1,,,5200.0,,,,


# STOP

База АМО c prefix

In [75]:
df_amo_temp = df_amo_temp.loc[df_amo_temp['prefix'] != '']

In [76]:
df_amo_temp['card_type'] = df_amo_temp['prefix'].apply(lambda x: 'electronic' if x[0] == 'V' else 'paper')

In [77]:
df_amo_temp.query('amo_id==26139623').head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,series,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
2611,26139623,Успешно_реализовано,11450,2023-11-28 13:06:38,2023-12-02 22:29:00,450.0,NaN,"Снегоходы_(Варианты:_1_снежик_на_1_час,_Бумажн...",11450.0,11000.0,11450,Снегоходы,SN1X_199622,NaN,SN1X_191480,NaN,NaN,NaN,NaT,,SN1X,199622,paper,,design1,,,0.0,,,,


In [78]:
df_amo_temp.query('корневая_сделка.notna()')

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,series,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
31,23459939,Успешно_реализовано,0,2023-01-13 11:41:15,2023-01-13 11:42:00,NaN,NaN,NaN,NaN,NaN,NaN,Искусственная_волна,VCW1T_2437849,#23459729,NaN,NaN,NaN,Погашен,2023-05-06,,VCW1T,2437849,electronic,,design1,sold_used,,0.0,,,,
38,23466787,Успешно_реализовано,0,2023-01-14 18:11:28,2023-01-18 10:48:00,NaN,NaN,NaN,NaN,NaN,NaN,Снегоходы,S_N2X226233,1715029001,NaN,NaN,NaN,Погашен,2023-03-03,,S,N2X226233,paper,,design1,sold_used,,0.0,,,,
103,23588587,Успешно_реализовано,0,2023-01-31 14:39:58,2023-01-31 14:40:00,NaN,NaN,NaN,NaN,NaN,NaN,Мото_Мотокросс,MT2_159461,#23586703,NaN,NaN,NaN,Погашен,2023-03-05,,MT2,159461,paper,,design1,sold_used,,0.0,,,,
104,23590237,Успешно_реализовано,8000,2023-01-31 18:10:26,2023-02-01 00:35:00,NaN,NaN,NaN,NaN,NaN,8000,Авто_Дрифт,VDRC2H_2598560,#23514709,NaN,NaN,NaN,NaN,NaT,,VDRC2H,2598560,electronic,,design1,,,0.0,,,,
126,23627015,Успешно_реализовано,0,2023-02-06 16:32:58,2023-02-07 18:19:00,NaN,NaN,NaN,NaN,NaN,NaN,Мото_Дрифт-трайк,DT60_282777,#23624295,NaN,NaN,NaN,Погашен,NaT,,DT60,282777,paper,,design1,sold_used,,0.0,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,25885815,Успешно_реализовано,0,2023-11-03 08:49:52,2023-11-03 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,Тир._Стрельба_из_боевого_оружия,VGU_4243962,#25885733,NaN,NaN,NaN,NaN,NaT,,VGU,4243962,electronic,,design1,,,0.0,,,,
2528,25939051,Успешно_реализовано,0,2023-11-09 16:31:32,2023-11-09 16:51:00,NaN,NaN,NaN,NaN,NaN,0,Багги,VBG_4327290,#25938825,NaN,NaN,NaN,NaN,NaT,,VBG,4327290,electronic,,design1,,,0.0,,,,
2552,25988317,Успешно_реализовано,0,2023-11-14 17:20:26,2023-11-14 17:29:00,NaN,NaN,NaN,NaN,NaN,0,Картинг,VKAT_4319612,#25988183,NaN,NaN,NaN,NaN,NaT,,VKAT,4319612,electronic,,design1,,,0.0,,,,
2580,26042895,Успешно_реализовано,0,2023-11-19 10:36:50,2023-11-20 12:51:00,NaN,NaN,NaN,NaN,NaN,NaN,Картинг,KAT_154768,#26042871,NaN,NaN,NaN,NaN,NaT,,KAT,154768,paper,,design1,,,0.0,,,,


Проверка:

In [79]:
pp = df_amo_temp.query('amo_id == 26139797')

In [80]:
pp

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,series,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
2612,26139797,Успешно_реализовано,0,2023-11-28 13:18:52,2023-12-02 22:30:00,NaN,NaN,NaN,NaN,NaN,0,Снегоходы,SN1X_184479,#26139623,NaN,NaN,NaN,NaN,NaT,,SN1X,184479,paper,,design1,,,0.0,,,,


Функция для определения sale_price для 

In [81]:
def insert_sale111(row):    
    v = row['sale_price']
    if row['sale_price'] == 0:
        amo_id = row['amo_id']
        pr = row['prefix']
        v = 0
        str1 = f'sale_price != 0 & prefix == "{pr}"'
        try:
            v = df_amo_temp.query(str1).iloc[-1]['sale_price']
        except:
            pass
        finally:
            return v
    return v

In [82]:
def insert_sale(row):    
    v = row['sale_price']
    #print(v)
    if row['sale_price'] == 0:
        d = row['дата_создания_сделки']
        amo_id = row['amo_id']
        pr = row['prefix']
        #v = 0
        str1 = f'sale_price != 0 & prefix == "{pr}"'
        try:
            #v = df_amo_temp.query(str1).iloc[-1]['sale_price']
            arr = df_amo_temp.query(str1)    
            arr['diff_date'] = arr['дата_создания_сделки'] - d
            arr = arr.sort_values(by='diff_date',ascending=True)
            arr.reset_index(drop= True , inplace= True)
            v = arr.iloc[0]['sale_price']
        except:
            pass
        finally:
            return v
    return v

In [83]:
df_amo_temp['sale_price'] = df_amo_temp.apply(insert_sale,axis=1)

Проверка:

In [84]:
pp = df_amo_temp.query('amo_id == 26139797')

In [85]:
pp

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,series,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
2612,26139797,Успешно_реализовано,0,2023-11-28 13:18:52,2023-12-02 22:30:00,NaN,NaN,NaN,NaN,NaN,0,Снегоходы,SN1X_184479,#26139623,NaN,NaN,NaN,NaN,NaT,,SN1X,184479,paper,,design1,,,5000.0,,,,


sale_price появился

Строки, оставшиеся без sale_price

In [86]:
dd0 = df_amo_temp.query('sale_price == 0').head(10)

In [87]:
dd0.shape

(8, 32)

In [88]:
dd0.head(10)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,series,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
310,23732921,Успешно_реализовано,0,2023-02-24 00:42:33,2023-02-24 00:42:00,NaN,NaN,NaN,NaN,NaN,NaN,Мото_Дрифт-трайк,VDT62_2660855,23732757,NaN,NaN,NaN,NaN,NaT,,VDT62,2660855,electronic,,design1,,,0.0,,,,
427,23908115,Успешно_реализовано,0,2023-03-20 21:14:58,2023-03-22 07:51:00,NaN,NaN,NaN,NaN,NaN,NaN,Аэротруба,AT6K_023008,#23906061,NaN,NaN,NaN,Погашен,NaT,,AT6K,023008,paper,,design1,sold_used,,0.0,,,,
568,24088395,Успешно_реализовано,12400,2023-04-16 20:02:43,2023-04-16 20:16:00,NaN,NaN,Подарочный_сертификатна_водные_лыжи_(Выберите_...,12400.0,12400.0,12400,Водные_лыжи,SKIN60_156264,NaN,NaN,NaN,TR212535,NaN,NaT,,SKIN60,156264,paper,,design1,,,0.0,,,,
1255,24596539,Успешно_реализовано,6800,2023-06-19 17:34:39,2023-06-21 23:01:00,NaN,NaN,Подарочный_сертификатна_вейк_за_катером_(Выбер...,6800.0,6800.0,6800,Вейксёрф_и_вейк_за_катером,ES130_164358,NaN,VWS30T_3324255DIS,NaN,NaN,NaN,NaT,,ES130,164358,paper,,design1,,,0.0,,,,
1288,24614375,Успешно_реализовано,7300,2023-06-21 17:57:54,2023-06-24 10:12:00,0.0,NaN,Подарочный_сертификатна_вейк_за_катером_(Выбер...,6800.0,6800.0,7300,Вейксёрф_и_вейк_за_катером,ES130_112723,NaN,NaN,NaN,TR2_220282,Погашен,2023-08-07,,ES130,112723,paper,,design1,sold_used,,0.0,,,,
1941,25010515,Успешно_реализовано,10950,2023-08-07 13:36:24,2023-08-08 15:50:00,450.0,NaN,Вейк_за_катером_(Варианты:_40_минут_Mastercraf...,10950.0,10500.0,10950,Вейксёрф_и_вейк_за_катером,WSN40_291295,NaN,NaN,NaN,TR2_153199,NaN,NaT,,WSN40,291295,paper,,design1,,,0.0,,,,
2036,25078171,Успешно_реализовано,16000,2023-08-14 17:08:31,2023-08-14 19:41:00,NaN,NaN,Полет_на_параплане_(Варианты:_Курс_«Первый_сам...,16000.0,16000.0,16000,Катер_без_капитана_(Летолодка),VLL3H_4147449,NaN,VPPT_3929836DIS,NaN,NaN,NaN,NaT,,VLL3H,4147449,electronic,,design1,,,0.0,,,,
2054,25090379,Успешно_реализовано,15000,2023-08-16 11:28:47,2023-08-16 12:18:00,0.0,NaN,"Самолёт_Cessna_182_(Варианты:_40_минут_полёта,...",15000.0,15000.0,15000,Мото_Дрифт-трайк,VDT62_4065301,NaN,VCS40_3929966DIS,NaN,NaN,Погашен,2023-01-09,,VDT62,4065301,electronic,,design1,sold_used,,0.0,,,,


Несколько записей так и не получили sale_price

In [89]:
df_amo_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2615 entries, 0 to 2621
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   amo_id                         2615 non-null   int64         
 1   этап_сделки                    2615 non-null   object        
 2   бюджет                         2615 non-null   int64         
 3   дата_создания_сделки           2615 non-null   datetime64[ns]
 4   sale_date                      2615 non-null   datetime64[ns]
 5   delivery_price                 1643 non-null   float64       
 6   стоимость_доставки             9 non-null      float64       
 7   products                       2447 non-null   object        
 8   price                          2447 non-null   float64       
 9   subtotal                       2447 non-null   float64       
 10  выручка_от_клиента             2493 non-null   object        
 11  направление_(вид_серти

Товары с префиксами, которые отсутствуют в Списке категорий

In [90]:
print(prefix_amo)

['AC60', 'AT10', 'AT15', 'AT4', 'AT4K', 'AT6', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'CS20', 'CS30', 'CS40', 'CS60', 'CW1', 'CW2', 'DRC2H', 'DT230', 'DT240', 'DT30', 'DT40', 'DT60', 'EB1X', 'EB2X', 'ED3H', 'EDC', 'EF30', 'EFL30', 'EFL60', 'ES130', 'F1', 'GC1', 'GC2', 'GR1', 'GR1X1', 'GR2', 'GU', 'GU2', 'HB20', 'HB30', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KR2', 'KR4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'LL1H', 'LL2H', 'MF', 'MF20', 'MF30', 'MF60', 'MT', 'MT2', 'MT5', 'MTT', 'P1', 'P1G', 'P1P', 'P2', 'P2G', 'PB', 'PB2', 'PB2X', 'PPP', 'PR20', 'PR30', 'PR40', 'PR60', 'PS', 'RB15', 'RB20', 'RB30', 'RC', 'RCV', 'RJ', 'RJ2', 'RTV', 'RXV', 'S', 'S2', 'SC', 'SC7', 'SG7', 'SH', 'SH2', 'SHH', 'SKI40', 'SKIN60', 'SKT1', 'SKT2', 'SKV', 'SN1X', 'SN1X1', 'SN2X', 'SN2X1', 'SP', 'SP2', 'ST', 'SU24', 'SU60', 'UN', 'VAC30', 'VAC60', 'VAT10', 'VAT15', 'VAT30', 'VAT4', 'VAT4K', 'VAT6', 'VAT6K', 'VBF180', 'VBF40', 'VBF90', 'VBG', 'VBGH', 'VBGK', 'VBGKH', 'VCS20', 'VCS30', 'VCS40', '

Замена кириллицы на латинские символы в prefix df_values

In [91]:
prefix_value = list(df_values['prefix'].unique())
prefix_value.sort()

In [92]:
len(prefix_value)

197

In [93]:
def replace_cirillic_latin2(row):
    v = row['prefix']
    #alf_ciril = 'ЕТОРАНКХСВМ'
    #alf_latin = 'ETOPAHKXCBM'
    new_v = ''
    for i in range(len(v)):
        simv = v[i]
        if simv in alf_ciril:
            pos = alf_ciril.index(simv)
            simv = alf_latin[pos]
        new_v += simv
    return new_v

In [94]:
df_values['prefix'] = df_values.apply(replace_cirillic_latin2,axis=1)

In [95]:
prefix_value = list(df_values['prefix'].unique())
prefix_value.sort()

Префиксы в списке покатегорий

In [96]:
print(prefix_value)

['2MWP24', '2WP24', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT6', 'AT60', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1x', 'EB2x', 'EB4x', 'ED3H', 'EDC', 'EFL30', 'EFL60', 'FR', 'FR2,', 'FRK', 'FRT', 'FRTK', 'GC1', 'GC2', 'GR1', 'GR2', 'GU', 'GU2', 'HB30', 'HB60', 'HY30', 'HY60', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KAT4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'KW', 'KW2', 'KYT', 'KYT2', 'LL1H', 'LL1HP', 'LL2H', 'LL2HP', 'LL3H', 'MB', 'MB2', 'MB2X', 'MEFL30', 'MEFL60', 'MF20', 'MF30', 'MF60', 'MHB60', 'MHY30', 'MHY60', 'MMF20', 'MMF30', 'MMF60', 'MSKI40', 'MSKI60', 'MSU24', 'MSU60', 'MT', 'MT10', 'MT5', 'MTT', 'MW&F', 'MWP15', 'MWP2H', 'MWP60', 'MWS1', 'MWS2', 'MWS25', 'MWS30', 'MWS40', 'MWSN1', 'MWSN2', 'MWSN25', 'MWSN40', 'OW2', 'OWG', 'OWI', 'PB', 'PB2'

In [97]:
len(prefix_value)

197

Префиксы, отсутсвующие в списке подкатегорий

In [98]:
#missing_prefix = set(prefix_value).difference(set(prefix_amo))
missing_prefix = list(set(prefix_amo)-(set(prefix_value)))
missing_prefix.sort()

In [99]:
print(missing_prefix)

['DT230', 'DT240', 'EB1X', 'EB2X', 'EF30', 'ES130', 'F1', 'GR1X1', 'HB20', 'KR2', 'KR4', 'MF', 'MT2', 'P1', 'P1G', 'P1P', 'P2', 'P2G', 'PS', 'S', 'S2', 'SC', 'SC7', 'SG7', 'SKIN60', 'SN1X1', 'SN2X1', 'SP', 'SP2', 'ST', 'UN', 'VAC30', 'VAC60', 'VAT10', 'VAT15', 'VAT30', 'VAT4', 'VAT4K', 'VAT6', 'VAT6K', 'VBF180', 'VBF40', 'VBF90', 'VBG', 'VBGH', 'VBGK', 'VBGKH', 'VCS20', 'VCS30', 'VCS40', 'VCS60', 'VCW1', 'VCW1T', 'VCW2', 'VCW2T', 'VDRC2H', 'VDT30', 'VDT40', 'VDT60', 'VDT62', 'VDTM', 'VEB1X', 'VEB2X', 'VED3H', 'VEF30', 'VEF60', 'VEFL', 'VEFL30', 'VFR', 'VFR2', 'VGC1', 'VGC2', 'VGR1', 'VGR2', 'VGU', 'VGU2', 'VHB20', 'VHB30', 'VHY30', 'VHY30T', 'VHY60', 'VHY60T30', 'VKA2', 'VKA4', 'VKA6', 'VKAT', 'VKAT2', 'VKAT4', 'VKV1X', 'VKV1X1', 'VKV2X1', 'VKVP1X', 'VKVP1X1', 'VKVP2X1', 'VLB1X', 'VLL1H', 'VLL2H', 'VLL3H', 'VMB', 'VMB2', 'VMB2X', 'VMF20', 'VMF30', 'VMF60', 'VMT', 'VMT2', 'VMT5', 'VMTT', 'VPB', 'VPP', 'VPP2', 'VPP2P', 'VPPP', 'VPPS', 'VPR20', 'VPR30', 'VPR60', 'VRC', 'VRCV', 'VRJ', 'VRJ

In [100]:
len(missing_prefix)

177

Товары, префиксы которых отсутствуют в списке категорий

In [101]:
df_amo_missing_prefix = df_amo_temp.query('prefix in @missing_prefix')

In [102]:
df_amo_missing_prefix.shape

(1970, 32)

Объединение датафреймов без префиксов и с отсутствующими префиксами и запись в файл

In [103]:
frames = [df_amo_without_prefix, df_amo_missing_prefix]

In [104]:
cert_w_missing_prefix = pd.concat(frames)

In [105]:
cert_w_missing_prefix.shape

(1977, 32)

In [106]:
cert_w_missing_prefix.to_csv('cert_w_missing_prefix.csv')

Запись отсутствующих в Списке подкатегорий prefix в файл

In [107]:
df_missing_prefix = pd.DataFrame({'prefix':missing_prefix})

In [108]:
df_missing_prefix.shape

(177, 1)

In [109]:
df_missing_prefix.head()

,prefix
0,DT230
1,DT240
2,EB1X
3,EB2X
4,EF30


In [110]:
df_missing_prefix.to_csv('missing_prefix.csv')

Подготовка товаров к объединению с Реестром подкатегорий

In [111]:
df_amo_temp = df_amo_temp.query('prefix in @prefix_value')

In [112]:
df_amo_temp.shape

(645, 32)

In [113]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,card_type,series,design_type,status,stock_status,sale_price,expiry_date,created_at,updated_at,deleted_at
4,23403789,Успешно_реализовано,5000,2023-01-03 21:56:43,2023-01-04 15:19:00,NaN,NaN,Квадроциклы_(Варианты:_2_квадрика_на_1_час_без...,9000.0,9000.0,5000,Квадроциклы,KVP1X1_138834,NaN,VKV2X1_2415531DIZ,NaN,NaN,Погашен,NaT,,KVP1X1,138834,paper,,design1,sold_used,,5000.0,,,,
8,23410089,Успешно_реализовано,15500,2023-01-05 14:17:46,2023-01-12 13:33:00,500.0,NaN,Вертолет_Robinson_R44_(Варианты:_20_минут_полё...,15500.0,15000.0,15500,Авиа_Вертолет,RB20_155687,NaN,NaN,NaN,NaN,Погашен,2023-03-07,,RB20,155687,paper,,design1,sold_used,,15000.0,,,,
9,23411393,Успешно_реализовано,8300,2023-01-05 19:46:14,2023-01-15 17:28:00,500.0,NaN,Аэротруба_(Варианты:_4_минуты_полёта_(для_1_че...,8300.0,7800.0,8300,Аэротруба,AT4_127813,NaN,NaN,NaN,NaN,Погашен,2023-08-10,,AT4,127813,paper,,design1,sold_used,,7800.0,,,,
13,23421175,Успешно_реализовано,10500,2023-01-08 14:05:08,2023-01-08 14:52:00,500.0,NaN,Квадроциклы_(Варианты:_2_квадрика_на_1_час_с_п...,10500.0,10000.0,10500,Квадроциклы,KVP2X1_155119,NaN,NaN,NaN,NaN,NaN,NaT,,KVP2X1,155119,paper,,design1,,,10000.0,,,,
15,23421479,Успешно_реализовано,7500,2023-01-08 15:17:05,2023-01-09 10:08:00,500.0,NaN,"Зимний_кайтинг_(Варианты:_2_часа_для_одного,_Б...",7500.0,7000.0,7500,Сноукайтинг,SKT1_113264,NaN,NaN,NaN,NaN,NaN,NaT,,SKT1,113264,paper,,design1,,,7000.0,,,,


Итоговые столбцы из АМО

In [114]:
lst_afm = ['id', 'amo_id',
               'series',
                'number',
                'prefix',
                'code',
                'card_type',
                'design_type',
                'status',
                'stock_status',
                'sale_price',
                'sale_date',
                'expiry_date',
                'card_used_date',
                'created_at',
                'updated_at',
                'deleted_at'
]

Итоговые столбцы из Реестра

In [115]:
lst_val = ['activity_value_id','prefix']

Датафреймы для объединения

In [116]:
df_amo_for_merge = df_amo_temp[lst_afm]

In [117]:
df_amo_for_merge.head()

,id,amo_id,series,number,prefix,code,card_type,design_type,status,stock_status,sale_price,sale_date,expiry_date,card_used_date,created_at,updated_at,deleted_at
4,,23403789,,138834,KVP1X1,KVP1X1_138834,paper,design1,sold_used,,5000.0,2023-01-04 15:19:00,,NaT,,,
8,,23410089,,155687,RB20,RB20_155687,paper,design1,sold_used,,15000.0,2023-01-12 13:33:00,,2023-03-07,,,
9,,23411393,,127813,AT4,AT4_127813,paper,design1,sold_used,,7800.0,2023-01-15 17:28:00,,2023-08-10,,,
13,,23421175,,155119,KVP2X1,KVP2X1_155119,paper,design1,,,10000.0,2023-01-08 14:52:00,,NaT,,,
15,,23421479,,113264,SKT1,SKT1_113264,paper,design1,,,7000.0,2023-01-09 10:08:00,,NaT,,,


In [118]:
df_value_for_merge = df_values[lst_val]

Объединение датафреймов

In [119]:
df_list = pd.merge(df_amo_for_merge, df_value_for_merge,how="inner",on='prefix')

In [120]:
df_list.head()

,id,amo_id,series,number,prefix,code,card_type,design_type,status,stock_status,sale_price,sale_date,expiry_date,card_used_date,created_at,updated_at,deleted_at,activity_value_id
0,,23403789,,138834,KVP1X1,KVP1X1_138834,paper,design1,sold_used,,5000.0,2023-01-04 15:19:00,,NaT,,,,122
1,,23561917,,151326,KVP1X1,KVP1X1_151326,paper,design1,sold_used,,5000.0,2023-01-27 09:42:00,,2023-08-10,,,,122
2,,23665615,,150930,KVP1X1,KVP1X1_150930,paper,design1,sold_used,,5000.0,2023-02-16 16:52:00,,2023-12-10,,,,122
3,,23794517,,68697,KVP1X1,KVP1X1_68697,paper,design1,,,10000.0,2023-03-04 12:40:00,,NaT,,,,122
4,,23825623,,259672,KVP1X1,KVP1X1_259672,paper,design1,sold_used,,5000.0,2023-03-10 19:42:00,,NaT,,,,122


In [121]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 645 non-null    object        
 1   amo_id             645 non-null    int64         
 2   series             645 non-null    object        
 3   number             645 non-null    object        
 4   prefix             645 non-null    object        
 5   code               645 non-null    object        
 6   card_type          645 non-null    object        
 7   design_type        645 non-null    object        
 8   status             645 non-null    object        
 9   stock_status       645 non-null    object        
 10  sale_price         632 non-null    float64       
 11  sale_date          645 non-null    datetime64[ns]
 12  expiry_date        645 non-null    object        
 13  card_used_date     90 non-null     datetime64[ns]
 14  created_at

Запись в файл

In [122]:
df_list.to_csv('list_of_goods_to_import.csv')